In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from IPython.display import clear_output

In [ ]:
!pip install gdown
!pip install transformers
!pip install sentencepiece
clear_output()

In [ ]:
!gdown https://drive.google.com/uc?id=1M4KnsDq_xeqACZQi30NLT1CYzqHl6b21

Downloading...
From: https://drive.google.com/uc?id=1M4KnsDq_xeqACZQi30NLT1CYzqHl6b21
To: /content/streets_cheb.csv
100% 27.2k/27.2k [00:00<00:00, 12.6MB/s]


In [ ]:
!wget https://clck.ru/YT75w -O data.zip
!unzip -u data.zip

--2021-10-30 07:54:09--  https://clck.ru/YT75w
Resolving clck.ru (clck.ru)... 213.180.204.221, 2a02:6b8::221
Connecting to clck.ru (clck.ru)|213.180.204.221|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://sba.yandex.net/redirect?url=https%3A%2F%2Fstorage.yandexcloud.net%2Fhackai-datasetrsv%2FArchive.zip&client=clck&sign=b351c9bcc3485cf0572cef79be48d6df [following]
--2021-10-30 07:54:09--  https://sba.yandex.net/redirect?url=https%3A%2F%2Fstorage.yandexcloud.net%2Fhackai-datasetrsv%2FArchive.zip&client=clck&sign=b351c9bcc3485cf0572cef79be48d6df
Resolving sba.yandex.net (sba.yandex.net)... 87.250.251.232, 87.250.250.232, 77.88.21.232, ...
Connecting to sba.yandex.net (sba.yandex.net)|87.250.251.232|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://storage.yandexcloud.net/hackai-datasetrsv/Archive.zip [following]
--2021-10-30 07:54:10--  https://storage.yandexcloud.net/hackai-datasetrsv/Archive.zip
Resolving stora

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.stem.snowball import SnowballStemmer
from tqdm import tqdm
from functools import lru_cache
from nltk.tokenize import word_tokenize
import nltk
import gc
from humanize import naturalsize
from copy import deepcopy
nltk.download('punkt')

stemmer = SnowballStemmer("russian") 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
@lru_cache(maxsize=None)
def stem(text):
    return stemmer.stem(text)

In [ ]:
model_checkpoint = 'cointegrated/rubert-base-cased-nli-twoway'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()

In [ ]:
df = pd.read_csv("/content/streets_cheb.csv")

In [ ]:
df["object_group"].unique()

array(['Улицы Чебоксар', 'Проспекты Чебоксар', 'Переулки Чебоксар',
       'Проезды Чебоксар', 'Бульвары Чебоксар', 'Шоссе Чебоксар'],
      dtype=object)

In [ ]:
df["object_group"] = df["object_group"].str.replace(" Чебоксар", "").str.strip()

In [ ]:
encode = {"Улицы": "Улица",
          "Проспекты": "Проспект",
          "Переулки": "Переулок",
          "Проезды": "Проезд",
          "Бульвары": "Бульвар",
          "Шоссе": "Шоссе"}

In [ ]:
df["object_group"] = df["object_group"].apply(lambda x: encode[x])

In [ ]:
df["address"] = df["object_group"] + " " + df["item_name"]

In [ ]:
classes = df["address"].to_list()
n_classes = len(classes)

In [ ]:
def predict_zero_shot(text, label_texts, model, tokenizer,
                      label='entailment', normalize=True):
    text = " ".join([stem(i) for i in word_tokenize(text)])
    tokens = tokenizer([text] * n_classes, label_texts,
                       truncation=True, return_tensors='pt',
                       padding=True, max_length=150)
    del text
    with torch.inference_mode():
        result = torch.softmax(model(**tokens.to(model.device)).logits, -1)
    proba = result[:, model.config.label2id[label]].cpu().numpy()
    if normalize:
        proba /= sum(proba)
    del result
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()
    return label_texts[proba.argmax()]

In [ ]:
predict_zero_shot('город Чебоксары улица К.Либкнехта дом 13', classes, model, tokenizer)

'Улица К.Либкнехта'

In [ ]:
df2 = pd.read_excel("/content/обращения граждан на портале народный контроль.xlsx").dropna(subset=["Текст обращения"])

In [ ]:
arr = []
for i in tqdm(df2["Текст обращения"]):
    arr.append(predict_zero_shot(i, classes, model, tokenizer))
    gc.collect()
    t = torch.cuda.get_device_properties(0).total_memory
    r = torch.cuda.memory_reserved(0)
    a = torch.cuda.memory_allocated(0)
    f = r-a # free inside reserved
    print(naturalsize(f))
    clear_output()

100%|██████████| 3137/3137 [1:32:26<00:00,  1.77s/it]


In [ ]:
df2["street"] = arr

In [ ]:
df2.to_csv("обращения_граждан_add_street.csv", index=False)

In [ ]:
for i in df2[["Текст обращения", "street"]].values[:10]:
    print(i)
    print("------------")

['Летом во дворе дома Филиппа Лукина 6 проводились работы по замене асфальтного покрытия, на время работ урны убрали. Почему до сих пор их не монтируют обратно?\nСоздание ссылки'
 'Улица Патриса Лумумбы']
------------
['После выборов в лесу за Кадетском сквером лежат агитационные листовки относительно Салаевой А. Л. Весь лес усыпан ими, как грибы. '
 'Улица Юннатская']
------------
['Опасный перекрёсток Академика Крылова / Пирогова. Светофоры несколько лет не работают, только установлены знаки пешеходный переход, в настоящее время из-за ремонта дороги даже нет разметки. Для пешеходов опасно переходить дорогу особенно когда темно, поскольку из-за активного нерегулиремого движения внимание расфокусируется. Ни один раз наблюдала, в том числе, когда сама переходила дорогу, как людям, в том числе с маленькими детьми, приходится махать руками вовремя перехода улицы, чтобы избежать дтп! Страшно. \nДля водителей- чтобы выехать на 30-ю дорогу с домов академика Крылова, приходится разворачиватьс

In [ ]:
ord("а"), ord("я")

(1072, 1103)

In [ ]:
"".join([chr(i) for i in range(1072, 1104)])

'абвгдежзийклмнопрстуфхцчшщъыьэюя'

In [ ]:
def is_have_digit(text):
    for i in text:
        if i.isdigit():
            return True
    return False

def is_correct(text):
    for i in text:
        if i not in "./,1234567890ёабвгдежзийклмнопрстуфхцчшщъыьэюя":
            return False
    return True

with_houses = []
houses = []
for idx, i in enumerate(df2[["Текст обращения", "street"]].values):
    text = i[0].lower()
    street = i[1].split()[-1].lower()
    t = text.find(street)
    if t != -1:
        temp = text[t:t + 100].split()[:2]
        try:
            if is_correct(temp[1]) and is_have_digit(temp[1]):
                with_houses.append(idx)
                houses.append(i[1] + " " + temp[1])
        except: pass

In [ ]:
new_df = deepcopy(df2.iloc[with_houses].copy())

In [ ]:
new_df["street"] = houses

In [ ]:
new_df.to_csv("обращения_граждан_full_address.csv", index=False)